In [5]:
import requests
from bs4 import BeautifulSoup
import os
import re

# URLs
base_url = 'https://kcserevision.com/'
target_url = 'https://kcserevision.com/grade-6-kicd-curriculum-designs-pdf/'

# Your login credentials
credentials = {
    'swpm_user_name': 'njoroge12',
    'swpm_password': 'njoroge'
}

# Create a session object
session = requests.Session()

# Headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

# Attempt to login
response = session.post(base_url, headers=headers, data=credentials)



In [6]:
response

<Response [200]>

In [14]:
import requests
from bs4 import BeautifulSoup
import os
import re

def extract_google_drive_links(session, url, headers):
    """Extracts Google Drive links, their titles, and the source URL from a given page URL."""
    response = session.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Add the modified source URL to the tuple, removing the base part
    drive_links = [(a.get_text(strip=True), a['href'], url.replace('https://kcserevision.com', '')) for a in soup.find_all('a', href=re.compile(r"https://drive\.google\.com/.*"))]
    return drive_links

# URLs
base_url = 'https://kcserevision.com/'
initial_url = 'https://kcserevision.com/'

# # Your login credentials
# credentials = {
#     'swpm_user_name': 'njoroge12',
#     'swpm_password': 'njoroge'
# }

# # Create a session object
# session = requests.Session()

# # Headers
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
# }

# # Attempt to login
# response = session.post(base_url, headers=headers, data=credentials)

# Check if login was successful by looking at cookies or response content
if 'swpm_session' in session.cookies:
    print("Logged in successfully.")
    
    # Get the initial page with the grade links
    response = session.get(initial_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all links that start with "GRADE "
    grade_links = [a['href'] for a in soup.find_all('a', href=True) if a.get_text().startswith("GRADE ")]

    # Directory to save the file
    processed_dir = '../data/processed/'
    if not os.path.exists(processed_dir):
        os.makedirs(processed_dir)

    # Filename for the output
    output_filename = os.path.join(processed_dir, 'google_drive_links.txt')
    
    # Open the file outside of the loop to append links from all grade pages
    with open(output_filename, 'w') as file:
        # Get the initial page with the grade links
        response = session.get(initial_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all links that start with "GRADE "
        grade_links = [a['href'] for a in soup.find_all('a', href=True) if a.text.startswith("GRADE ")]

        # Process each grade link
        for grade_url in grade_links:
            drive_links = extract_google_drive_links(session, grade_url, headers)
            # Save the links, titles, and source URL to the file
            for title, link, source_url in drive_links:
                file.write(f"{source_url}/{title}: {link}\n")  # Concatenate the source URL with title and link
            print(f"Processed {grade_url}")
            
    print(f"Google Drive links from all grade pages saved to {output_filename}")

else:
    print("Failed to login.")


Logged in successfully.
Processed https://kcserevision.com/grade-1-2-3-4-cbc-lesson-plans/
Processed https://kcserevision.com/grade-1-2-3-4-5-6-7-8-cbc-revision/
Processed https://kcserevision.com/2024-grade-7-8-cbc-junior-secondary-resources/
Processed https://kcserevision.com/grade-8-junior-secondary-schemes-of-work/
Processed https://kcserevision.com/grade-8-junior-secondary-exams/
Processed https://kcserevision.com/grade-8-junior-secondary-assignments/
Processed https://kcserevision.com/grade-8-junior-secondary-notes/
Processed https://kcserevision.com/grade-8-junior-secondary-lesson-plans/
Processed https://kcserevision.com/grade-8-curriculum-designs-kicd/
Processed https://kcserevision.com/2024-grade-8-assessment-and-scoresheet/
Processed https://kcserevision.com/grade-8-term-1-2-3-opener-mid-end-exams/
Processed https://kcserevision.com/2024-grade-7-schemes-of-work-term-1-2-3/
Processed https://kcserevision.com/2024-jss-grade-7-term-1-2-3-exams/
Processed https://kcserevision.co

In [3]:
# Check if login was successful by looking at cookies or response content
if 'swpm_session' in session.cookies:
    print("Logged in successfully.")

    # Get the page with the files
    response = session.get(target_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all Google Drive links and their titles
    drive_links = [(a.get_text(strip=True), a['href']) for a in soup.find_all('a', href=re.compile(r"https://drive\.google\.com/.*"))]

    # Directory to save the file
    processed_dir = '../data/processed/'
    if not os.path.exists(processed_dir):
        os.makedirs(processed_dir)

    # Save the links and titles to a file
    with open(os.path.join(processed_dir, 'google_drive_links.txt'), 'w') as file:
        for title, link in drive_links:
            file.write(f"{title}: {link}\n")
        print(f"Google Drive links saved to {os.path.join(processed_dir, 'google_drive_links.txt')}")

else:
    print("Failed to login.")


Logged in successfully.
Google Drive links saved to ../data/processed/google_drive_links.txt


In [17]:
import gdown
import os
import re

# Function to create subdirectories based on grade and thematic area
def create_subdirectory(download_dir, filename):
    grade = re.search(r'GRADE \d+', filename, re.IGNORECASE)
    thematic_area = re.search(r'(SCHEMES|EXAMS|ASSIGNMENTS|NOTES|LESSON PLANS|CURRICULUM|ASSESSMENT AND SCORESHEET|TERM \d+)', filename, re.IGNORECASE)
    
    subdirectory = 'Miscellaneous'
    if grade and thematic_area:
        subdirectory = f"{grade.group(0).capitalize()}/{thematic_area.group(0).capitalize()}"
    
    subdirectory_path = os.path.join(download_dir, subdirectory)
    if not os.path.exists(subdirectory_path):
        os.makedirs(subdirectory_path)
    
    return subdirectory_path

# Paths
file_path = '../data/processed/google_drive_links_v1.txt'
download_directory = '../data/downloads/'
download_log_path = '../data/interim/downloaded_files.log'
error_log_path = '../data/interim/error_files.log'

# Regular expression pattern to extract file ID from Google Drive link
pattern = r'drive\.google\.com/file/d/([a-zA-Z0-9_-]+)/view'

# Ensure download directory exists
if not os.path.exists(download_directory):
    os.makedirs(download_directory)

# Load downloaded files log
if os.path.exists(download_log_path):
    with open(download_log_path, 'r') as log_file:
        downloaded_files = log_file.read().splitlines()
else:
    downloaded_files = []

# Load error log
if os.path.exists(error_log_path):
    with open(error_log_path, 'r') as error_file:
        failed_downloads = error_file.read().splitlines()
else:
    failed_downloads = []

# Process links
with open(file_path, 'r') as file:
    for line in file:
        if line in downloaded_files or line in failed_downloads:
            continue  # Skip if already processed

        match = re.search(pattern, line)
        if match:
            try:
                file_id = match.group(1)
                filename = line.split(':')[0].strip()
                download_link = f'https://drive.google.com/uc?id={file_id}'
                
                subdirectory_path = create_subdirectory(download_directory, filename)
                output_path = os.path.join(subdirectory_path, filename)
                
                gdown.download(download_link, output_path, quiet=False)
                
                with open(download_log_path, 'a') as log_file:
                    log_file.write(f"{line}\n")
            except Exception as e:
                print(f"Failed to download {line.strip()}. Error: {e}")
                with open(error_log_path, 'a') as error_file:
                    error_file.write(f"{line}\n")
        else:
            print(f"Could not extract file ID from line: {line}")


Failed to download GRADE 6 TERM 1 AGRICULTURE SCHEMES.docx: https://drive.google.com/file/d/11Csc06oFzmsRvNZ7vFOuRv2GPfmzQZ7g/view?usp=drive_web. Error: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=11Csc06oFzmsRvNZ7vFOuRv2GPfmzQZ7g

but Gdown can't. Please check connections and permissions.
Failed to download GRADE 6 TERM 1 ART CRAFT SCHEMES.docx: https://drive.google.com/file/d/1_BoTRTCUIqRHpVBmU2tkWo4i4f4nuspA/view?usp=drive_web. Error: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1_BoTRTCUIqRHpVBmU2tkWo4i4f4nuspA

but Gdown can't. Please check connections a

KeyboardInterrupt: 